This notebook creates a **One-Pager Summary** from an ISBN using OpenAI's API.


In [19]:
#@markdown # Enter the required information for the summarization process.
#@markdown ---

#@markdown **Step 1:** Enter API Key and ISBN

open_api_key = '' #@param {type:"string"}
isbn = '' #@param {type:"string"}

In [20]:
!pip install openai
import json
import pprint
from openai import OpenAI
import time
from google.colab import files
import io
from concurrent.futures import ProcessPoolExecutor
import requests
import os

In [21]:
def add_space_between_sentences(text):
    save = []
    for i,v in enumerate(text):
        if v in ['.', '?', '!'] and i+1 < len(text) and text[i+1].isupper():
            #text = text.replace(char, char + ' ')
            save.append(i)

    counter = 0
    for i,v in enumerate(save):

        if i == 0:
            text = text[:v+1] + ' ' + text[v+1:]
            counter += 1
        else:
            text = text[:v+counter+1] + ' ' + text[v+counter+1:]
            counter += 1
    return text

def text_cleaner(summary):
    # summary = summary.replace('\t', '')
    # summary = summary.replace('\n', '')
    if summary:
        summary = " ".join(summary.split())

    return summary

model = 'gpt-4-1106-preview'

def prompt_model(text, model=model, open_api_key=open_api_key):
    """Send text to OpenAI model and return the response."""
    try:
        start = time.time()
        client = OpenAI(api_key=open_api_key)
        print("Prompting model...")
        chat_completion = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": text}]
        )
        response = chat_completion.choices[0].message.content
        print("Response received -- Time taken: {:.2f} seconds".format(time.time() - start))
        return response
    except Exception as e:
        print(f"Failed to prompt model: {e}")
        raise

def generate_one_pager(title, authors, metadata):
    prompt = f"Give me a long summary of {title} by {authors} given this metadata: {metadata}"
    result = prompt_model(prompt, model=model)
    print(result)
    return result

def read_all_files_in_directory(directory):
    files = os.listdir(directory)
    return files

def get_google_book_id(isbn):
# Changed the API URL from a custom endpoint to the Google Books API:
# Previous URL: 'https://book-summarizer-api-develop.up.railway.app/google_books_query/isbn:{isbn}'
# New URL: 'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}'
# Reason for change:
# The Google Books API is more reliable and widely supported, providing robust and direct access to extensive book data without the need for third-party services.
    try:
        url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}'
        headers = {'accept': 'application/json'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            results = response.json()
            if 'items' in results and results['items']:
                google_book_id = results['items'][0]['id']
                print(f"Google Book ID {google_book_id} has been fetched for {isbn}.")
                return results['items'][0]['volumeInfo']
            else:
                print("No books found for the given ISBN.")
                return None
        else:
            print(f"Failed to fetch book details, status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching book details: {e}")
        return None

def prepare_one_pager(isbn):
    google_book_metadata = get_google_book_id(isbn)
    if google_book_metadata:
        try:
            one_pager = generate_one_pager(google_book_metadata['title'], google_book_metadata.get('authors', ['Unknown author']), google_book_metadata)
            return one_pager
        except Exception as e:
            print(f"Failed to generate one pager: {e}")
            return "Error in generating summary"
    else:
        print("Metadata not found, cannot generate one pager.")
        return "Metadata not found"

def write_to_file(file_name, text):
    with open(file_name, 'w') as file:
        file.write(text)
    print(f"File saved as {file_name}")



In [22]:
#@markdown **Step 3:** Generate One Pager Summary and Download

#@markdown Ensure that the ISBN is correctly entered at the beginning of this notebook. Once verified, click the **"Play"** button on the left of this cell.

one_pager_content = prepare_one_pager(isbn)

output_filename = f'{isbn}_one_pager.txt'
download_link = write_to_file(output_filename, one_pager_content)
files.download(output_filename)

Google Book ID OprNBAAAQBAJ has been fetched for 1626813582.
Prompting model...
Response received -- Time taken: 24.17 seconds
"Exponential Organizations" by Salim Ismail, Yuri van Geest, and Mike Malone is a seminal work that examines the emerging business entities known as Exponential Organizations, or ExOs, that are reshaping the landscape of the corporate world through their unparalleled ability to grow and innovate at a pace far surpassing that of traditional companies.

At the core of the book is the observation that certain organizations can achieve performances that are not just incrementally better but exponentially superior—often 10 times more effective than their peers. This radical advantage is largely credited to their aptness to harness cutting-edge technological assets and innovative organizational strategies.

The authors argue that these Exponential Organizations thrive by employing a new organizational structure that is suitable for the modern, fast-paced, technologic

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>